In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 24.4 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 


In [2]:
from pyomo.environ import * 

In [3]:
import numpy as np

In [4]:
coef = np.loadtxt('lab7_ex2_lp_ip_coef.txt', delimiter=',')

In [5]:
# create a model
model_lab7_ex2 = ConcreteModel()

In [6]:
# N=500
# M=5

In [7]:
coef.shape

(6, 501)

In [8]:
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])

Num rows: 6
Num cols: 501


In [9]:
N = coef.shape[1]-1 #variables

In [10]:
M = coef.shape[0]-1 #constraints

In [11]:
print(N,M)

500 5


In [12]:
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(500,)
[ 801.  956.  512.  823.  740.  752.  936.  536.  851.  399.  515.  660.
  523.  229.  637.  609.  663.  561.  516.  709. 1060.  405.  564.  692.
  524.  992.  745.  860.  638.  801.  777.  757.  722.  785.  321.  943.
  469.  541.  673.  716.  686. 1188.  969.  804.  728.  790.  741.  790.
  545.  591. 1091.  707.  475.  808.  750.  707.  556.  573.  562. 1057.
  750.  919.  511.  440.  638.  865.  588.  891.  731.  548.  437. 1015.
  878. 1017.  824.  993.  905.  600.  664.  890.  732.  696.  703.  348.
  690.  392. 1090. 1170.  799. 1191.  561.  840.  555.  918.  698.  687.
  735.  572.  926.  599.  469.  486.  973.  676.  817.  504.  388.  685.
  596.  878.  426.  862.  695.  542.  746.  694.  885.  524.  694.  935.
  715.  810.  978.  443.  318.  621.  805.  511. 1279.  832.  974.  737.
  857.  756.  595.  830.  526.  968.  974. 1279.  770.  980.  435.  881.
  858.  964.  454. 1068.  956.  874.  872.  623.  609.  623.  649.  620.
  850.  522.  999.  959.  593.  624.  488.  

In [13]:
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(5, 500)
[[702. 840. 513. ... 474. 707. 625.]
 [246. 994. 938. ... 734. 423. 691.]
 [799. 228.  10. ...  13. 231. 440.]
 [463. 654. 167. ... 574. 711. 678.]
 [252. 735. 586. ... 738. 947. 798.]]


In [14]:
# We can segregate the RHS of constraints into a separate array
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[173725. 190019. 186926. 192737. 183645.]


In [15]:
#Since the variables have the same upper bound and lower bound, we can simply use scalars to store them
lower_bound = 0
upper_bound = 1

In [16]:
# set of column indices: since we have N variables, we can take column indices to be the list [0,1,2,...,N-1]
col_indices = np.arange(N)
print(col_indices)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [17]:
#declare the decision variables in the model
model_lab7_ex2.x = Var(col_indices, domain=Binary)

In [18]:
# set of row indices: since we have M contraints, we can take row indices to be the list [0,1,2,...,M-1]
row_indices = np.arange(M)
print(row_indices)

[0 1 2 3 4]


In [19]:
# create a ConstraintList to hold multiple constraints
model_lab7_ex2.constraints = ConstraintList()

In [20]:
for i in row_indices:
  model_lab7_ex2.constraints.add(summation(constr_coef[i],model_lab7_ex2.x) <= constr_rhs[i])

In [21]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model_lab7_ex2.x[j].setlb(lower_bound)
  model_lab7_ex2.x[j].setub(upper_bound)

In [22]:
# add the model objective
model_lab7_ex2.objective = Objective(expr = summation(obj_coef,model_lab7_ex2.x), sense=maximize)

In [23]:
model_lab7_ex2.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  500 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 15

In [24]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 159447 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [25]:
opt_cbc = SolverFactory('cbc')

In [26]:
result = opt_cbc.solve(model_lab7_ex2)
print(result)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 295828.0
  Upper bound: 295828.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 500
  Number of binary variables: 500
  Number of integer variables: 500
  Number of nonzeros: 500
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 66.26
  Wallclock time: 70.18
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 347044
      Number of created subproblems: 347044
    Black box: 
      Number of iterations: 1407089
  Error rc: 0
  Time: 70.27342700958252
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [27]:
# display solution
print('\nObjective = ', model_lab7_ex2.objective())


Objective =  295828.0


In [28]:

result.solver.time

70.27342700958252

$ \huge{3.}$  The optimal objective function value is 295828.0

$ \huge{4.}$ The MILP takes 72.48865962028503 seconds.

In [29]:
model_lab7_ex2.x.domain=NonNegativeReals

In [30]:
result = opt_cbc.solve(model_lab7_ex2)
print(result)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 295896.377
  Upper bound: 295896.377
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 501
  Number of nonzeros: 500
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 18
  Error rc: 0
  Time: 0.02424001693725586
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [31]:
# display solution
print('\nObjective = ', model_lab7_ex2.objective())


Objective =  295896.37697236


In [32]:
result.solver.time

0.02424001693725586

$ \huge{5.}$ The objective function values of the optimal solution is 295896.37697236

$ \huge{6.}$ The LP takes 0.027211427688598633 seconds. This time is much lesser than the time taken to solve the MILP.

$ \huge{7.}$ For MILP: \\
Branch and bound: \\
      Number of bounded subproblems: None \\
      Number of created subproblems: None \\
      \\
For LP: \\
Branch and bound: \\
      Number of bounded subproblems: None \\
      Number of created subproblems: None \\